In [2]:
from langchain import OpenAI, VectorDBQA, LLMChain
from langchain.prompts import PromptTemplate
import os
from dotenv import load_dotenv
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader

In [3]:
import sys
sys.path.append(r"C:\Users\ELAFACRB1\Codice\GitHub\media-chat-service\src\embedding")
from utils import ChromaDBManager, LangChainAI, QDrantDBManager, EmbeddingFunction

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

In [5]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
embedding = EmbeddingFunction('openAI').embedder
vectore_store=qdrantClient = QDrantDBManager(
    url="http://ec2-18-209-145-26.compute-1.amazonaws.com:6333/dashboard",
    port=6333,
    collection_name="rio-rag-platform",
    vector_size=1536,
    embedding=embedding,
    record_manager_url=r"sqlite:///C:\Users\ELAFACRB1\Codice\GitHub\media-chat-service\src\embedding\record_manager_cache.sql"
)
vectore_store_client=vectore_store.vector_store

c:\Users\ELAFACRB1\venvs\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
c:\Users\ELAFACRB1\venvs\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


Collection rio-rag-platform already exists!


In [6]:
COLLECTION_NAME="rio-rag-platform"

In [7]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    chain_type="stuff",
    retriever=vectore_store_client.as_retriever(),
    return_source_documents=True,
)

In [8]:
query='what the role on product owner in Safe? '

In [9]:
qa_chain(query)

c:\Users\ELAFACRB1\venvs\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'query': 'what the role on product owner in Safe? ',
 'result': 'The role of the Product Owner in SAFe (Scaled Agile Framework) is responsible for connecting with the customer, contributing to the vision and roadmap, managing and prioritizing the team backlog, supporting the team in delivering value, and getting and applying fast feedback. The Product Owner works closely with the Agile Team to ensure that the product or solution being developed meets the needs of the customer and aligns with the overall vision and goals of the organization.',
 'source_documents': [Document(page_content='efficiencies over timeFOR REFERENCE ONLY 2-13 Workbook 38 © Scaled Agile, Inc. For personal use only - Rosario Laface - Nov 16, 2023 (ID:0056T00000949fqQAA)  © Scaled Agile. Inc. © Scaled Agile, Inc.  2.2 The Scrum Master / Team Coach and  Product Owner roles 2-16 © Scaled Agile. Inc. Team Charter Agile Purpose Success Measures Team Members Team Type & ResponsibilitiesDefinition of Done Distinctive Com

Parent document

In [10]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

In [15]:
from langchain_community.vectorstores import Qdrant

In [16]:
Qdrant

langchain_community.vectorstores.qdrant.Qdrant

In [11]:
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

In [12]:
store = InMemoryStore()

In [18]:
retriever = ParentDocumentRetriever(
    vectorstore=vectore_store.vector_store,
    docstore=store,
    child_splitter=child_splitter,
)

In [20]:
from langchain.document_loaders import RSSFeedLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
urls = ["https://medium.com/feed/tag/quantum-computing"]
loader = RSSFeedLoader(urls=urls)
data = loader.load()

In [21]:
retriever.add_documents(data, ids=None)

In [24]:
retriever

ParentDocumentRetriever(vectorstore=<langchain_community.vectorstores.qdrant.Qdrant object at 0x000001AD239F8C90>, docstore=<langchain.storage.in_memory.InMemoryBaseStore object at 0x000001AD247C1E50>, child_splitter=<langchain.text_splitter.RecursiveCharacterTextSplitter object at 0x000001AD251CFF50>)

In [22]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    chain_type="stuff",
    retriever=vectore_store_client.as_retriever(),
    return_source_documents=True,
)

In [23]:
qa_chain('whats the most recent computer for quantum computing?')

{'query': 'whats the most recent computer for quantum computing?',
 'result': "The most recent computer for quantum computing is the Sycamore processor developed by Google. It made headlines in 2019 when Google claimed to have achieved quantum supremacy with this processor. However, it's important to note that the field of quantum computing is rapidly evolving, and new advancements are being made by various companies and research institutions.",
 'source_documents': [Document(page_content='The Future of Quantum Computing', metadata={'authors': [], 'description': 'Are you tired of waiting ages for data processing to complete? Frustrated by the limitations of conventional computing power? If so, you’re not alone. In a world where data is king, speed and…', 'doc_id': '7ed35252-4320-4062-a7e2-aada649aa0b1', 'feed': 'https://medium.com/feed/tag/quantum-computing', 'language': 'en', 'link': 'https://medium.com/@krupalu.datta/quantum-supremacy-redefining-speed-and-efficiency-in-data-processin

**Multi-Query Retriever**

If users are asking questions that are complex and require multiple pieces of distinct information to respond

In [26]:
query='what could be an use of quantum computing in SAFe methodology?'

In [32]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI
import logging

In [28]:
llm = ChatOpenAI(temperature=0)

In [33]:
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [29]:
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectore_store_client.as_retriever(), 
    llm=llm
)

In [35]:
unique_docs = retriever_from_llm.get_relevant_documents(query=query)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. How can quantum computing be applied within the context of SAFe methodology?', '2. In what ways could quantum computing enhance the implementation of SAFe methodology?', '3. What potential benefits does quantum computing offer for integrating with SAFe methodology practices?']


In [38]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    chain_type="stuff",
    retriever=retriever_from_llm,
    return_source_documents=True,
)

In [39]:
qa_chain(query)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. How can quantum computing be applied within the context of SAFe methodology?', '2. In what ways could quantum computing enhance the implementation of SAFe methodology?', '3. What potential benefits can quantum computing bring to the utilization of SAFe methodology?']


{'query': 'what could be an use of quantum computing in SAFe methodology?',
 'result': 'Quantum computing could potentially be used in SAFe methodology to enhance the development of solutions by improving computational power and efficiency. Quantum algorithms could help in optimizing processes, such as accelerating the development of new pharmaceuticals or solving complex optimization problems in logistics, finance, and manufacturing. This could lead to more efficient and effective implementation of SAFe practices, ultimately improving the quality and speed of solution delivery within the framework.',
 'source_documents': [Document(page_content='The Future of Quantum Computing', metadata={'authors': [], 'description': 'Are you tired of waiting ages for data processing to complete? Frustrated by the limitations of conventional computing power? If so, you’re not alone. In a world where data is king, speed and…', 'doc_id': '7ed35252-4320-4062-a7e2-aada649aa0b1', 'feed': 'https://medium.co

In [40]:
qa_chain('whats a PI?')

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can you explain the concept of PI?', '2. What is the definition of PI?', '3. Could you provide information about PI?']


{'query': 'whats a PI?',
 'result': 'PI stands for Planning Interval. It is a cadence-based event that serves as the heartbeat of the ART (Agile Release Train), aligning all teams on the ART to a shared mission and vision. PI Planning helps in establishing personal communication across all team members and stakeholders, aligning development of business goals with the business context, identifying dependencies, fostering cross-team and cross-ART collaboration, providing architecture and Lean UX guidance, matching demand to capacity, and enabling faster decision-making.',
 'source_documents': [Document(page_content='dependencies 4-24Planning Interval (PI) Planning is a cadence-based event that serves as the  heartbeat of the ART, aligning all teams on the ART to a shared mission  and Vision.What is PI Planning? © Scaled Agile. Inc. Duration Video: The Power of PI Planning 3 https://bit.ly/Video - PowerofPIPlanning min 4-23 Workbook 85 © Scaled Agile, Inc. For personal use only - Rosario 

**Output parser**

In [58]:
from langchain.output_parsers import PydanticOutputParser
from langchain_core.output_parsers import JsonOutputParser

from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import OpenAI

In [42]:
model = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.0)

In [59]:
from typing import List
class News(BaseModel):
    topic: str = Field(description="The topic of the news")
    title: List[str] = Field(description="list of the titles of the news")
    description: List[str] = Field(description="list of long text of the news")

    # You can add custom validation logic easily with Pydantic.
    # @validator("setup")
    # def question_ends_with_question_mark(cls, field):
    #     if field[-1] != "?":
    #         raise ValueError("Badly formed question!")
    #     return field

In [60]:
parser = JsonOutputParser(pydantic_object=News)

In [61]:
prompt = PromptTemplate(
    template="Answer the user query \n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)


In [62]:
prompt_and_model = prompt | model
output = prompt_and_model.invoke({"query": "Tell me the last 5 news about quantum computing."})
parser.invoke(output)

{'topic': 'Quantum Computing', 'title': ['1. ']}